##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [2]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 947.4 kB/s eta 0:00:00


In [3]:
import keras_tuner as kt

## Download and prepare the dataset

In this tutorial, you will use the Keras Tuner to find the best hyperparameters for a machine learning model that classifies images of clothing from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Load the data.

In [4]:
import pandas as pd
df = pd.read_csv('admission_data.csv')

In [ ]:
df.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65


In [5]:
X = df.iloc[:,0:7]
y = df.iloc[:,-1]

In [ ]:
X.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0


In [ ]:
y.head()

0    0.92
1    0.76
2    0.72
3    0.80
4    0.65
Name: Chance of Admit , dtype: float64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.25)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
X_train

array([[-0.43126246,  0.4405822 , -0.95902733, ..., -0.5154475 ,
        -0.08651715, -1.11602374],
       [-0.06681531, -0.70563017, -0.09034315, ..., -1.60136217,
        -1.50073985, -1.11602374],
       [ 1.57319686,  2.07802844,  1.64702519, ...,  1.65638185,
         2.14296333,  0.89603829],
       ...,
       [-1.34238033, -1.36060866, -1.8277115 , ..., -1.60136217,
        -2.21617016, -1.11602374],
       [-0.70459782, -0.37814092, -0.95902733, ...,  0.57046717,
        -1.50073985, -1.11602374],
       [-0.24903888, -0.21439629, -0.95902733, ...,  0.02750984,
        -0.55237875, -1.11602374]])

In [9]:
X_test

array([[ 1.57319686,  1.42304995,  0.77834102,  0.62370211,  0.02750984,
         1.59391217,  0.89603829],
       [-0.24903888,  0.11309295,  0.77834102,  1.12777171,  0.57046717,
         0.76201646,  0.89603829],
       [-0.1579271 , -0.37814092, -0.95902733, -1.39257627, -1.05840483,
        -1.55065359, -1.11602374],
       [-0.43126246,  0.27683758, -0.09034315, -0.38443708, -0.5154475 ,
         0.17968947, -1.11602374],
       [ 0.84430256,  0.76807145, -0.09034315,  0.11963252, -0.5154475 ,
         0.77865438,  0.89603829],
       [ 1.11763792,  0.60432683,  0.77834102,  1.12777171,  0.57046717,
         1.07813683,  0.89603829],
       [-2.43572177, -1.19686404, -1.8277115 , -1.89664586, -1.60136217,
        -1.70039482, -1.11602374],
       [-1.43349212, -0.21439629,  0.77834102, -0.88850667, -0.5154475 ,
        -0.18634464, -1.11602374],
       [ 0.11540826,  0.27683758, -0.09034315,  0.11963252,  0.57046717,
         1.06149892,  0.89603829],
       [ 0.29763184,  0.7680

In [10]:
def model_builder(hp):
  model = keras.Sequential()
  #model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
  hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
  hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units1, activation='relu',input_shape=(7,)))
  model.add(keras.layers.Dense(units=hp_units2, activation='relu'))
  model.add(keras.layers.Dense(units=hp_units3, activation='relu'))

  model.add(keras.layers.Dense(1,activation='linear'))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.MeanSquaredError( name="mean_squared_error"),
                metrics=['mse'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [11]:
tuner = kt.Hyperband(model_builder,
                     objective='val_mse',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

The Hyperband tuning algorithm uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket. The algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer.

Create a callback to stop training early after reaching a certain value for the validation loss.

In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [13]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])



Trial 30 Complete [00h 00m 04s]
val_mse: 0.0037916931323707104

Best val_mse So Far: 0.0037916931323707104
Total elapsed time: 00h 01m 08s


In [14]:
# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

for i in ['units1','units2','units3','learning_rate']:
  print(i," ",best_hps.get(i))

units1   288
units2   256
units3   480
learning_rate   0.01


## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [15]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)



Epoch 1/50
10/10 [==============================] - 1s 26ms/step - loss: 2.4451 - mse: 2.4451 - val_loss: 0.5690 - val_mse: 0.5690
Epoch 2/50
10/10 [==============================] - 0s 11ms/step - loss: 0.1840 - mse: 0.1840 - val_loss: 0.0569 - val_mse: 0.0569
Epoch 3/50
10/10 [==============================] - 0s 9ms/step - loss: 0.0482 - mse: 0.0482 - val_loss: 0.0334 - val_mse: 0.0334
Epoch 4/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.0263 - val_mse: 0.0263
Epoch 5/50
10/10 [==============================] - 0s 9ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.0100 - val_mse: 0.0100
Epoch 6/50
10/10 [==============================] - 0s 10ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.0073 - val_mse: 0.0073
Epoch 7/50
10/10 [==============================] - 0s 11ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0078 - val_mse: 0.0078
Epoch 8/50
10/10 [==============================] - 0s 10ms/step - loss: 0.0057 - mse

In [16]:
eval_result = model.evaluate(X_test, y_test)
print("[test loss, test accuracy]:", eval_result)

4/4 [==============================] - 0s 4ms/step - loss: 0.0043 - mse: 0.0043
[test loss, test accuracy]: [0.004279609303921461, 0.004279609303921461]
